# Tuning Machine Learning models in Spark

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## ML Pipelines in Spark

ML model training and tuning often represents running the same steps once and again. Often, we run the same steps with small variations in order to evaluate combinations of parameters. 

In order to make this use case a lot easier, Spark provides the [Pipeline](https://spark.apache.org/docs/2.3.0/ml-pipeline.html) abstraction.

A Pipeline represents a series of steps in the processing of a dataset. Each step is a Transformer or an Estimator. The whole Pipeline is an Estimator, so we can .fit the whole pipeline in one step. When we do that, the steps'  .fit and .transform methods will be called in turn.

![pipelineestimator](https://spark.apache.org/docs/2.3.0/img/ml-Pipeline.png)

![PipelineModel](https://spark.apache.org/docs/2.3.0/img/ml-PipelineModel.png)

## Example: predicting flight delays

We'll be using the same [Transtats'](https://www.transtats.bts.gov/) OTP performance data] from way back when. Remember it?

It's a table that contains all domestic departures by US air air carriers that represent at least one percent of domestic scheduled passenger revenues, with data on each individual departure including [Tail Number](https://en.wikipedia.org/wiki/Tail_number), departure delay, origin, destination and carrier.


### Load the data

Opening .zip files in Spark is a bit of a pain. For now, let's just decompress the file we want to read. When we are ready to expand the processing to the cluster, we will need to do [this](https://stackoverflow.com/questions/28569788/how-to-open-stream-zip-files-through-spark).

In [77]:
df = spark.read.csv('On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_12.csv.gz', 
                    header=True, inferSchema=True)

df = df[['Year', 'Month', 'DayofMonth', 'FlightDate', 'DayOfWeek',
         'Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline', 'Origin', 
         'OriginCityName', 'OriginStateName', 'Dest', 'DestCityName', 'DestStateName',
         'DepTime', 'DepDelay', 'AirTime', 'Distance']]

df

DataFrame[Year: int, Month: int, DayofMonth: int, FlightDate: timestamp, DayOfWeek: int, Reporting_Airline: string, Tail_Number: string, Flight_Number_Reporting_Airline: int, Origin: string, OriginCityName: string, OriginStateName: string, Dest: string, DestCityName: string, DestStateName: string, DepTime: int, DepDelay: double, AirTime: double, Distance: double]

### Drop nas

There are only a few departures for which any of the columns of interest contains null values. The most expedient way to handle them is to just drop them, since they won't make much of a difference.

In [78]:
df.dropna()

DataFrame[Year: int, Month: int, DayofMonth: int, FlightDate: timestamp, DayOfWeek: int, Reporting_Airline: string, Tail_Number: string, Flight_Number_Reporting_Airline: int, Origin: string, OriginCityName: string, OriginStateName: string, Dest: string, DestCityName: string, DestStateName: string, DepTime: int, DepDelay: double, AirTime: double, Distance: double]

NA-related functions are grouped in a .na attribute of DataFrames.

In [79]:
df = df.na.drop()

## Feature extraction and generation of target variable

The departing hour is the most important factor in delays, so we need to calculate it from the departure time. Since the input file uses a funny format for times, Spark has interpreted them as floats:

In [80]:
df.select('DepTime').show(5)

+-------+
|DepTime|
+-------+
|   1048|
|    638|
|   1710|
|   1318|
|    953|
+-------+
only showing top 5 rows



In [81]:
from pyspark.sql import functions, types

minutes = (df['DepTime'] / 100).cast(types.IntegerType()) * 60 + df['DepTime'] % 100

df2 = df.withColumn('DepMinutes', minutes )
df2

DataFrame[Year: int, Month: int, DayofMonth: int, FlightDate: timestamp, DayOfWeek: int, Reporting_Airline: string, Tail_Number: string, Flight_Number_Reporting_Airline: int, Origin: string, OriginCityName: string, OriginStateName: string, Dest: string, DestCityName: string, DestStateName: string, DepTime: int, DepDelay: double, AirTime: double, Distance: double, DepMinutes: int]

We will also generate a binary target variable. The aviation industry considers a flight delayed when it departs more than 15 minutes after its scheduled departure time, so we will use that. We will create it as an integer, since that is what the learning algorithms expect.

In [82]:
data = df2.withColumn('Delayed', (df['DepDelay'] > 15).cast(types.ByteType())) 
data

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/ml/wrapper.py", line 40, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'
Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/ml/wrapper.py", line 40, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'RandomForestClassifier' object has no attribute '_java_obj'


DataFrame[Year: int, Month: int, DayofMonth: int, FlightDate: timestamp, DayOfWeek: int, Reporting_Airline: string, Tail_Number: string, Flight_Number_Reporting_Airline: int, Origin: string, OriginCityName: string, OriginStateName: string, Dest: string, DestCityName: string, DestStateName: string, DepTime: int, DepDelay: double, AirTime: double, Distance: double, DepMinutes: int, Delayed: tinyint]

In order to make the training times manageable, let's pick only 10% of the data to train.

In [83]:
train, test = data.randomSplit([1.,9.])

## Handle different fields in different ways

We have features of at least three kinds:

* Numeric continuous fields, which we can use as input to many algorithms as they are. In particular, decision trees can take continuous variables with any value as input, since they only look for the cutoff point that most increases the homogeneity of the resulting groups. In contrast, if we were using a logistic regression with regularization, for example, we would need to first scale the variables to have comparable magnitudes.

* There are fields which we will treat as categorical variables, but which are already integers. These need to be one-hot encoded.

* Finally, there are several categorical variables that are encoded as strings. These need to be one-hot encoded, but OneHotEncoder requires numeric input. Therefore, we will need to apply a StringIndexer to each of them before one-hot encoding.

We have generated the list of names of columns that have dataType string with a list comprehension, rather than hard-coding it, but it is just like the other ones.

In [87]:
train.schema.fields[0].name

'Year'

In [88]:
categorical_fields = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'Reporting_Airline', 
                      'Origin', 'OriginCityName', 'OriginStateName', 
                      'Dest', 'DestCityName', 'DestStateName']

string_fields = [field.name for field in train.schema.fields if field.dataType == types.StringType()]

continuous_fields = ['Distance', 'DepMinutes']

target_field = 'Delayed'

## Handling categorical fields

Let's do the processing of just one field first, as an example. Then we will process the rest.

### StringIndexer 

A [StringIndexer](https://spark.apache.org/docs/2.2.0/ml-features.html#stringindexer) is an estimator that takes a single string field, then produces a transformer that codifies said field as numeric labels that are fit for feeding to a one-hot encoding. 

We need to specify an input column, an output column, and a way to handle invalids. In this case, invalids are values that the indexer has not seen during fitting but that the transformer finds during processing. Its values are 'error' (the default), which is pretty self-explanatory, 'skip', which drops them, and 'keep', which is what we want. It will assign all unseen labels to a single category index.

In [90]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

indexerEstimator = StringIndexer(inputCol='Reporting_Airline', outputCol='Reporting_AirlineIndex')
indexerModel = indexerEstimator.fit(train)
indexerModel

StringIndexer_e78933aee869

In [91]:
indexed = indexerModel.transform(train)
indexed

DataFrame[Year: int, Month: int, DayofMonth: int, FlightDate: timestamp, DayOfWeek: int, Reporting_Airline: string, Tail_Number: string, Flight_Number_Reporting_Airline: int, Origin: string, OriginCityName: string, OriginStateName: string, Dest: string, DestCityName: string, DestStateName: string, DepTime: int, DepDelay: double, AirTime: double, Distance: double, DepMinutes: int, Delayed: tinyint, Reporting_AirlineIndex: double]

### OneHotEncoderEstimator

A [OneHotEncoderEstimator](https://spark.apache.org/docs/latest/ml-features#onehotencoderestimator) generates a n-1 length vector column for an n-category column of category indices. 

We need to specify an input and an output column.

In [92]:
onehotEstimator = OneHotEncoderEstimator(inputCols=['Reporting_AirlineIndex'], 
                                           outputCols=['Reporting_AirlineOneHot'],
                                           handleInvalid='keep')

onehotEstimator

OneHotEncoderEstimator_7e05e5336541

In [93]:
onehotModel = onehotEstimator.fit(indexed)
processed = onehotModel.transform(indexed)

processed[['Reporting_Airline', 'Reporting_AirlineIndex', 'Reporting_AirlineOneHot']]\
    .sample(fraction=.001)\
    .show(10)

+-----------------+----------------------+-----------------------+
|Reporting_Airline|Reporting_AirlineIndex|Reporting_AirlineOneHot|
+-----------------+----------------------+-----------------------+
|               WN|                   0.0|         (17,[0],[1.0])|
|               YX|                   6.0|         (17,[6],[1.0])|
|               B6|                   5.0|         (17,[5],[1.0])|
|               WN|                   0.0|         (17,[0],[1.0])|
|               MQ|                   7.0|         (17,[7],[1.0])|
|               WN|                   0.0|         (17,[0],[1.0])|
|               AA|                   2.0|         (17,[2],[1.0])|
|               EV|                  13.0|        (17,[13],[1.0])|
|               OH|                   8.0|         (17,[8],[1.0])|
|               YV|                  11.0|        (17,[11],[1.0])|
+-----------------+----------------------+-----------------------+
only showing top 10 rows



### SparseVectors

The vectors produced by OneHotEncoder will each have only one non-zero value, but can potentially be very long. An efficient way to represent them is therefore a SparseVector, and that is what OneHotEncoder generates. 

A SparseVector is a data structure that only stores the length of the vector, a list of positions, and a list of values. All other values are assumed to be 0s.

This way, a vector like the following, with lenght 15 and non-zero values only on positions 3 and 9:

```python
[0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0]
```

can be compactly expressed as

```python
(15, [3, 9], [6.0, 4.0])
```

## Let's build our first Pipeline!

Our pipeline consists of a number of StringIndexers, followed by one OneHotEncoder, followed by a VectorAssembler, with a RandomForestClassifier at the end.

A Spark Pipeline is a single Estimator. We build it secifying the stages it comprises, and then we are ready to .fit it in one go. This will save us a lot of trouble, since we don't need to fit and transform each stage individually.

In [94]:
from pyspark.ml import Pipeline

pipeline = Pipeline()

### StringIndexer stages

We only need to StringIndex some of the fields. We are going to build the input and output column names programatically.


In [95]:
indexers = [ StringIndexer(inputCol=field, outputCol=field + 'Index', handleInvalid='keep') for field in string_fields ]
indexers

[StringIndexer_5c5082e9722a,
 StringIndexer_b06f93212e64,
 StringIndexer_22c725f6043f,
 StringIndexer_d9cbf9efd1fd,
 StringIndexer_fc70ff73bcc9,
 StringIndexer_ebbab9c313a4,
 StringIndexer_969b9026eb1b,
 StringIndexer_82cb89c78ce4]

### OneHotEncoderEstimator

One OneHotEncoderEstimator can handle all categorical columns. We are also going to build it programatically

In [98]:
inputCols = [ field for field in categorical_fields 
                     if field not in string_fields] + \
            [ field + 'Index' for field in string_fields]


onehotencodedcols = [ field + 'OneHot' for field in categorical_fields 
                                          if field not in string_fields] +\
                     [ field + 'OneHot' for field in string_fields]


onehotEstimator = OneHotEncoderEstimator(inputCols=inputCols, outputCols=onehotencodedcols)
onehotEstimator

OneHotEncoderEstimator_91dff1ad7d53

### VectorAssembler

Once we have generated our features, we can assemble them into a single features column, together with the continuous_fields.

In [99]:
assembler = VectorAssembler(inputCols=onehotencodedcols + continuous_fields,
                            outputCol='features')

### RandomForestClassifier

Aaaaand we are ready to do some Machine Learning! We'll use a RandomForestClassifier to try to predict delayed versus non delayed flights, a binary classification task.

In [100]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol='features',
                                    labelCol='Delayed')

### Pipeline!

Now that we have all the stages, we are finally ready to put them together into a single Estimator, our Pipeline.

In [101]:
pipeline = Pipeline(stages= indexers + [onehotEstimator] + [assembler] + [classifier]) 

Now that we have gone to the trouble of building our Pipeline, fitting it and using it to predict the probabilty of delay on unseen data is as easy as using a single Estimator:

In [102]:
pipeline_model = pipeline.fit(train)

PipelineModel_c604bdb1134d

In [104]:
pipeline_model

PipelineModel_c604bdb1134d

In [117]:
result = pipeline_model.transform(test).cache()
result[['features', 'rawPrediction', 'probability', 'prediction' ]].show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(8806,[2031,2067,...|[16.4727818974929...|[0.82363909487464...|       0.0|
|(8806,[2031,2067,...|[16.5244298956837...|[0.82622149478418...|       0.0|
|(8806,[2031,2067,...|[16.4727818974929...|[0.82363909487464...|       0.0|
|(8806,[2031,2067,...|[16.4727818974929...|[0.82363909487464...|       0.0|
|(8806,[2031,2067,...|[16.4727818974929...|[0.82363909487464...|       0.0|
|(8806,[2031,2067,...|[16.7445647059967...|[0.83722823529983...|       0.0|
|(8806,[2031,2067,...|[16.5244298956837...|[0.82622149478418...|       0.0|
|(8806,[2031,2067,...|[16.7445647059967...|[0.83722823529983...|       0.0|
|(8806,[2031,2067,...|[16.5605708815836...|[0.82802854407918...|       0.0|
|(8806,[2031,2067,...|[16.5244298956837...|[0.82622149478418...|       0.0|
|(8806,[2031

In [125]:
accuracy = result.select((result['prediction'] == result['Delayed']).cast(types.IntegerType()).alias('right!')).agg(functions.mean('right!'))
accuracy.show()

+------------------+
|       avg(right!)|
+------------------+
|0.8286197915679413|
+------------------+



In [128]:
delayed = result.select((result['Delayed']).cast(types.IntegerType()).alias('right!')).agg(functions.mean('right!'))
delayed.show()

+-------------------+
|        avg(right!)|
+-------------------+
|0.17138020843205864|
+-------------------+



## Evaluating and tuning our Pipeline

Probably the most interesting use of Spark Pipelines is quickly (in terms of coding time) evaluating many combinations of hyperparameters to feed our model and choosing the best ones. For that, we can use a TrainValidationSplit or a CrossValidator. The CrossValidator will generally perform better, but it will take several times as much. I'm using here the TrainValidationSplit because the API is the same.

In [130]:
from pyspark.ml.tuning import TrainValidationSplit, CrossValidator


TrainValidationSplit(estimator=pipeline)

TrainValidationSplit_ccbbdbb7b4ab

### Params and Evaluators

In order to evaluate different sets of parameters, we need a) the set of parameters to iterate through and b) a metric to compare the results. 

The first element is represented by ParamMaps, which we build with a ParamGridBuilder, and the second by an Evaluator that needs to be specific to the relevant task.

In [132]:
RandomForestClassifier()

VectorAssembler_aa5539f31924

In [136]:
from pyspark.ml.tuning import ParamGridBuilder

builder = ParamGridBuilder()

builder.addGrid(classifier.maxBins, [16, 32, 64])
builder.addGrid(classifier.numTrees, [10, 20, 50])
builder.addGrid(classifier.maxDepth, [3, 5, 10])

param_map = builder.build()

In [138]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='Delayed')

We now have all the elements in place to perform our fit:

In [139]:
split = TrainValidationSplit(estimator=pipeline, 
                             estimatorParamMaps=param_map, 
                             evaluator=evaluator)

And now we can predict on the rest of the flights and compare them with reality:

In [ ]:
%%time

results_of_exploration = split.fit(train)

### Let's have a look

We are now ready to compare our predictions with reality. Do these features have any predictive power at all?

Not bad, considering we have not performed any feature engineering at all!

### Further Reading

https://spark.apache.org/docs/latest/ml-tuning.html

https://stackoverflow.com/questions/28569788/how-to-open-stream-zip-files-through-spark